In [1]:
from differential_evolution import optimize
from timer import Timer
import torch
import torch.nn.functional as F
from time import sleep
from itertools import permutations
from fastprogress.fastprogress import master_bar

def pairwise_distances(A,B):
    # Both A and B should be (bs,n,d) where n is some number of points and d is dimension (e.g. 2 for R^2)
    w = (A[:,:,None] - B[:,None])
    d2 = (w*w).sum(dim = -1)
    return torch.sqrt(d2)

def inverse_perm(P):
    Q = [0]*len(P)
    for i,p in enumerate(P):
        Q[p] = i
    return Q

def add_points(T, A):
    T = torch.tensor(T,device=A.device,dtype=A.dtype)
    bs = A.shape[0]
    W = torch.cat([T.repeat(bs,1,1),A],dim=1)
    return W

def too_close_loss(D):
    β = (D < 0.01).double()
    return (D*β).sum(dim=1)

def distances_too_similar(D):
    U = pairwise_distances(D[...,None],D[...,None])
    u = U.shape[1]

    i = torch.arange(u,device=U.device)
    U[:,i,i] = 1
    U = U.view(U.shape[0],-1)
    β = (U < 0.001).double()
    return (U*β).sum(dim=1)

class PermCost():
    def __init__(self, P, num_points_left_side, default_points):
        self.P = P
        self.n = num_points_left_side
        self.T = default_points
        
    def __call__(self, A):
        T,P,n = self.T,self.P,self.n
        
        W = add_points(T,A)
        B,C = W[:,:n], W[:,n:]
        D = pairwise_distances(B,C)
        bs = D.shape[0]
        D = D.view(bs,-1)
        S,_ = D.sort(dim=1)
        
        #raise "BLA"
        P.to(D.device)
        
        return torch.sum(torch.abs(D[:,P] - S), dim=1) + too_close_loss(D) + distances_too_similar(D)

class PermCostNoDefaults():
    def __init__(self, P, num_points_left_side):
        self.P = P
        self.n = num_points_left_side
        
    def __call__(self, A):
        P,n = self.P,self.n
        P.to(A.device)
        
        B,C = A[:,:n], A[:,n:]
        D = pairwise_distances(B,C)
        bs = D.shape[0]
        D = D.view(bs,-1)
        S,_ = D.sort(dim=1)
        
        #raise "BLA"
        
        
        return torch.sum(torch.abs(D[:,P] - S), dim=1) + too_close_loss(D) + distances_too_similar(D)
    

In [3]:
n,m = 5,5
d = 4

shuffles = 0
epochs = 12000
num_populations = 128
pop_size = 32
bs = num_populations*pop_size
device = 'cuda'

simulaciones = 1000

num_malas = 0
malas = []

mb = master_bar(range(simulaciones))

for _ in mb:
    p = torch.randperm(n*m,device=device)
    initial_pop = torch.randn((bs,n+m,d),device=device).double()
    value, x = optimize(PermCostNoDefaults(p,n), 
                        num_populations=num_populations, initial_pop=initial_pop, 
                        epochs=epochs, 
                        shuffles=shuffles,
                        proj_to_domain=lambda x: torch.clamp(x,-30,30), #+ torch.randn_like(x)*0.00001, 
                        use_cuda=False,
                        break_at_cost=0,
                        mb=mb
                       )
    try:
        sleep(0.2)
    except KeyboardInterrupt:
        raise
    
    if value != 0:
        print(f"BAD permutation: {p}, best value: {value}\n")
        malas.append(p)

BAD permutation: tensor([21,  5, 15, 13,  1,  8, 24, 17, 20, 14, 19,  6,  2,  7, 12, 18, 10,  3,
        16,  0, 22,  9, 11,  4, 23], device='cuda:0'), best value: 0.04568202815248945

BAD permutation: tensor([ 1,  3, 20, 22, 15, 18,  7, 24, 10, 11, 21, 13, 17,  4, 14, 23, 19,  8,
         0, 16,  5,  9, 12,  6,  2], device='cuda:0'), best value: 0.004981351764465458

BAD permutation: tensor([11, 13,  8, 22, 15,  2,  3,  5,  4, 16, 14,  9, 19, 24, 17, 10, 20,  1,
        23, 18,  6, 12,  7, 21,  0], device='cuda:0'), best value: 0.00358597089585011

BAD permutation: tensor([ 3, 10, 14, 13, 18, 16,  4, 24, 17, 12, 22,  9, 21, 23,  5,  8,  6,  2,
         1, 15, 19,  7, 11,  0, 20], device='cuda:0'), best value: 0.002662903815796369

BAD permutation: tensor([24, 13, 11,  0, 18,  2,  1,  5,  4, 23,  8,  7, 21, 12, 22,  6, 19, 16,
        10, 20,  9, 17, 15,  3, 14], device='cuda:0'), best value: 0.03380645099913249

BAD permutation: tensor([17,  8,  9, 22, 16,  3,  4,  5,  1, 21, 12, 23, 

In [4]:
malas

[tensor([21,  5, 15, 13,  1,  8, 24, 17, 20, 14, 19,  6,  2,  7, 12, 18, 10,  3,
         16,  0, 22,  9, 11,  4, 23], device='cuda:0'),
 tensor([ 1,  3, 20, 22, 15, 18,  7, 24, 10, 11, 21, 13, 17,  4, 14, 23, 19,  8,
          0, 16,  5,  9, 12,  6,  2], device='cuda:0'),
 tensor([11, 13,  8, 22, 15,  2,  3,  5,  4, 16, 14,  9, 19, 24, 17, 10, 20,  1,
         23, 18,  6, 12,  7, 21,  0], device='cuda:0'),
 tensor([ 3, 10, 14, 13, 18, 16,  4, 24, 17, 12, 22,  9, 21, 23,  5,  8,  6,  2,
          1, 15, 19,  7, 11,  0, 20], device='cuda:0'),
 tensor([24, 13, 11,  0, 18,  2,  1,  5,  4, 23,  8,  7, 21, 12, 22,  6, 19, 16,
         10, 20,  9, 17, 15,  3, 14], device='cuda:0'),
 tensor([17,  8,  9, 22, 16,  3,  4,  5,  1, 21, 12, 23,  7, 20, 18,  6, 11, 14,
         19, 10, 13, 15,  0, 24,  2], device='cuda:0'),
 tensor([21,  3, 20, 13,  7, 17, 16, 24,  1,  2,  0, 10, 14, 22,  9, 15,  4,  5,
          8, 18, 19, 23, 12,  6, 11], device='cuda:0'),
 tensor([18,  1,  0,  8,  9,  7, 16, 14, 

In [5]:
print(f"% de malas: {len(malas)/simulaciones}")

% de malas: 0.077


In [19]:
n,m = 3,4
d = 2

shuffles = 0
epochs = 10000
num_populations = 64
pop_size = 32
bs = num_populations*pop_size
device = 'cuda'

num_malas = 0

mb = master_bar(malas)
muy_malas = []
for p in mb:
    #p = torch.randperm(n*m,device=device)
    initial_pop = torch.randn((bs,n+m,d),device=device).double()*5
    value, x = optimize(PermCostNoDefaults(p,n), 
                        num_populations=num_populations, initial_pop=initial_pop, 
                        epochs=epochs, 
                        shuffles=shuffles,
                        proj_to_domain=lambda x: torch.clamp(x,-20,20) + torch.randn_like(x)*0.00001, 
                        use_cuda=False,
                        break_at_cost=0,
                        mb=mb
                       )
    try:
        sleep(0.1)
    except KeyboardInterrupt:
        raise
    
    if value != 0:
        print(f"BAD permutation: {p}, best value: {value}\n")
        muy_malas.append(p)

BAD permutation: tensor([ 5,  2,  4,  6,  8,  7, 10, 11,  3,  1,  0,  9], device='cuda:0'), best value: 7.939253231370458e-05

Interrupting! Returning best found so far
BAD permutation: tensor([ 9,  8,  7,  3,  1,  4,  2,  6,  0, 10,  5, 11], device='cuda:0'), best value: 7.527103568663282

Interrupting! Returning best found so far
BAD permutation: tensor([10, 11,  1,  6,  5,  9,  8,  4,  0,  3,  2,  7], device='cuda:0'), best value: 6.902296556285036

Interrupting! Returning best found so far
BAD permutation: tensor([ 3,  4,  9,  8,  7,  1, 10,  6, 11,  2,  5,  0], device='cuda:0'), best value: 10.232459973565884

Interrupting! Returning best found so far
BAD permutation: tensor([ 5,  6, 10,  8,  0,  1, 11,  7,  4,  3,  9,  2], device='cuda:0'), best value: 9.704542617986583

Interrupting! Returning best found so far
BAD permutation: tensor([ 2,  6,  9,  4,  1,  8, 11,  3, 10,  7,  0,  5], device='cuda:0'), best value: 9.033198326775231

Interrupting! Returning best found so far
BAD p

In [ ]:
bad_ones = []
n,m = 3,4
def_pts = [[-1,0],[1,0]]
device = 'cuda'
#X = [[0] + list(p) for p in permutations(range(1,n*m))]
X =  [[0, 1, 2, 3, 5, 10, 6, 7, 11, 9, 8, 4], [0, 1, 2, 3, 5, 9, 10, 11, 7, 4, 8, 6]]

for i,p in enumerate(X):
    perm = torch.tensor(p,device=device)
    initial_pop = torch.randn((512,n+m-len(def_pts),2),device=device).double()
    
    value, x = optimize(PermCost(perm,n,def_pts), 
                        num_populations=32, initial_pop=initial_pop, 
                        epochs=9000, 
                        shuffles=0,
                        proj_to_domain=lambda x: torch.clamp(x,-20,20), #+ torch.randn_like(x)*0.00001, 
                        use_cuda=False,
                        break_at_cost=0
                       )
    if value != 0:
        print(f"BAD permutation: {p}, best value: {value}\n")
        W = add_points(def_pts,x[None])
        B,C = W[:,:n], W[:,n:]
        print(f"{100.*i/len(X):.3f}%: Perm {p} CANNOT be realized. Closest I found: \n{B}¸and \n{C}\n Distance matrix:\n{pairwise_distances(B,C)}")
        
        bad_ones += [p]
        try:
            sleep(0.2)
        except KeyboardInterrupt:
            raise
    else:
        W = add_points(def_pts,x[None])
        B,C = W[:,:n], W[:,n:]
        print(f"{100.*i/len(X):.3f}%: Perm {p} can be realized with \n{B}¸and \n{C}\n Distance matrix:\n{pairwise_distances(B,C)}")
            
        
print("DONE! Bad ones for now: ", bad_ones)

In [ ]:
bad_ones = []
n,m = 3,4
def_pts = [[-1,0],[1,0]]
device = 'cuda'
X = get_perms(n*m)

In [ ]:
len(X)

In [ ]:
perm = perm = torch.tensor(X[0],device=device)
f=PermCost(perm,3,def_pts)

In [ ]:
initial_pop = torch.randn((8,n+m-len(def_pts),2),device=device).double()

In [ ]:
initial_pop.shape

In [ ]:
f(initial_pop)

In [ ]:
for i,p in enumerate(X):
    perm = torch.tensor(p,device=device)
    initial_pop = torch.randn((1024,2*n-len(def_pts),2),device=device).double()
    
    value, x = optimize(PermCost(perm,3,def_pts), 
                        num_populations=8, initial_pop=initial_pop, 
                        epochs=2000, 
                        shuffles=0,
                        proj_to_domain=lambda x: torch.clamp(x,-20,20), #+ torch.randn_like(x)*0.00001, 
                        use_cuda=False,
                        break_at_cost=0
                       )
    if value != 0:
        print(f"BAD permutation: {p}, best value: {value}\n")
        bad_ones += [p]
        try:
            sleep(0.2)
        except KeyboardInterrupt:
            raise
    else:
        if i%256 == 0:
            W = add_points(def_pts,x[None])
            B,C = W[:,:n_left], W[:,n_left:]
            print(f"{100.*i/len(X):.3f}%: Perm {p} can be realized with \n{B}¸and \n{C}\n Distance matrix:\n{pairwise_distances(B,C)}")
            
        
print("DONE! Bad ones for now: ", bad_ones)

In [ ]:
D = torch.rand(2,4)

In [ ]:
U = pairwise_distances(D[...,None],D[...,None])

u = U.shape[1]

i = torch.arange(u,device=U.device)

U[:,i,i] = 1

Φ = (U < 0.0001).double()

In [ ]:
(Φ*U)#.sum(1)

In [ ]:
D = torch.rand(1,2,3)

In [ ]:
bs = D.shape[0]
D = D.view(bs,-1)
_,I = D.sort(dim=1)

In [ ]:
I

In [ ]:
_

In [ ]:
D[:,I]

In [ ]:
(0, 1, 2, 3, 4, 6, 5, 7, 8)

In [ ]:
A = torch.tensor([[ 1.9503, 14.5673],
        [-0.3314, -5.1933],
        [-5.7467, -1.6767],
        [ 4.1168, -4.4749]])[None]

In [ ]:
A,c = A[:,1:],A[:,0]
D = disttoABC(A,c)
#_,I = D.sort(dim=1)

In [ ]:
D

In [ ]:
def d(a,b):
    x = a[0]-b[0]
    y = a[1]-b[1]
    return x*x+y*y

In [ ]:
A.shape

In [ ]:
p,q,r = A[0]

In [ ]:
p

In [ ]:
d(p,c[0])

In [ ]:
d(q,c[0])

In [ ]:
d(r,c[0])